In [2]:
from openai import OpenAI
import json
import csv
import re
from lcpartition import *
import os
# import dotenv
import ast
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from validationClass3 import *

In [3]:
#Converts to json to string
def convertToString(path): 
    with open(path, 'r') as file:
        fullText = json.load(file)
    return fullText


def listToString(list):
    str = ""
    for x in list:
        str += '\n' + x
    return str


def separateNames(page, titleIndices):
    cleanedPage = ""
    for index, content in enumerate(page):
        # Check if the current index is in titleIndices to add a newline
        if index in titleIndices:
            cleanedPage += '\n'  # Add a newline before the title/content
            cleanedPage += ' '
        cleanedPage += content
        cleanedPage += ' '
    return cleanedPage


def splitPage(page, titleIndices):
    midpoint = titleIndices[(len(titleIndices)) // 2]
    partition1 = page[0:midpoint]
    partition2 = page[midpoint-2:]
    return partition1, partition2


#Uses spacy to detect the named entities on the page and labels them
def tagNames1(rawData, titleIndices):
    names = []  # This will store the extracted names
    counter = 0
    for text in rawData:
      if counter in titleIndices:
        names.append(rawData[counter:counter+2])
      counter+=1
    return names

def outputToList(output):
    strippedOutput = output.rstrip(" \"\"\"")
    list = strippedOutput.strip("[")
    list = list.strip("]")
    list = list.split(",")
    return list

In [4]:
client = OpenAI()

#Currently using json instead of text files for the input
OCRPath = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\yellowBookStuff\\fileReads\\standardizedSegmentResults75.json"
filePath = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\yellowBookStuff\\fileReads\\standardizedSegmentResults75.json"
titleIndices = []
blockedList = []
nlp = spacy.load("en_core_web_sm")
rawData = pathToString(filePath)
singlePage = rawData[3]
fullresponse = ""

In [40]:
#This file does not run in a loop and clean every page in the text, it only takes one. Used for testing.
processedPath =  convertToString(OCRPath)
processedPathStr = str(processedPath[7])
print("this is the string we are using right now: ", processedPathStr)

this is the string we are using right now:  ['Administration continued', '', 'Assistant Vice President, Campus Life and Dean of', 'Students Dr. Robert D. Hradsky.................... (202) 885-33 1%', 'E-mail: hradsky@american.edu', 'Education: Loyola Col (MD) BBA; Vermont; Pennsylvania 2007 EdD', 'Assistant Vice President, Development', 'Abbey Silberman Fagin................ 00-0 ee eee (202) 885-34]', 'E-mail: acs@american.edu', 'Assistant Vice President, Risk Management and Safety', 'Services. Patricia Kelshian............ 00.00 cee eee (202) 8$5- 3284', 'E-mail: pat@american.edu mo,', 'Chief Information Security Officer', 'Cathy Hubbs, CISSP CISA................. Lee eaee (202) 885-3994', 'E-mail: hubbs@american.edu .', 'Education: Strayer U BS; George Mason MEd', 'Executive Director, Career Center Gihan Fernando...... (202) 885-1829', 'Senior Director, Diversity and Inclusion Tiffany Speaks . . (202) 885-365', 'Education: Duke BA: Stanford MA; Columbia MS', 'Director, Athletics and

In [6]:
clean_message = """\
You have been assigned the role of a data extractor. Your task is to process unstructured text and clean the text of \
unnecessary information about faculty members mentioned in the text. The text will contain details about many faculty members at U.S. Universities.

### Your Responsibilities:
1. **Read and Analyze Text:**
   Carefully read the provided unstructured text and identify any occurences of phone numbers and email addresses within the text. \
   Remove these phone numbers and email addresses, and nothing else.


2. **Handle Ambiguities:**
   In cases where the text contains ambiguities or conflicting information, use your best judgement to resolve them and \
extract the most accurate and relevant information.

3. **Validate Information:**
   Ensure that the deleted information was only phone numbers and email addresses, and nothing else. Also ensure that any handled \
   ambiguities were necessary to be altered.

   ### Steps to Process:
1. **Removal:**
   Remove all occurrences of phone numbers and emails within the text.

2. **Structure:**
   Ensure that you did not change the overall structure of the original text. This means do not add any extra formatting, or extra information. \
   Return the data as it was, except with removals and ambiguity fixes.


### Preparation for Output:

Remember, accuracy and attention to detail are crucial in this task. Ensure that the structured information you extract is a \
true representation of the details mentioned in the unstructured text. Your role is pivotal in converting raw, unstructured data into meaningful, \
structured information that can be easily understood and utilized by other systems.
"""

In [184]:
# response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     # response_format = {"type": "text_object"},
#     messages=[
#         {"role": "system", "content": clean_message},
#         {"role": "system", "content": "Ensure that you return information for EVERY individual within the dataset."},
#         {"role": "system", "content": "Do not return a json"},
#         {"role": "user", "content": processedPathStr},
#     ],
#     temperature=.1,
#     # top_p=1,
#     frequency_penalty=0,
#     presence_penalty=0,
#     # seed=1001
# )


# cleaned_text = response.choices[0].message.content
# print(cleaned_text)




#Remove extraneous information
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are part of a data processing pipeline. You will be given a dataset containing university faculty members, and information about these faculty. Your task is to remove any phone numbers and email addresses that appear (Or any string containing at least 7 numbers), any strings of characters that are gibberish, and any non-ASCII characters."},
    {"role": "user", "content": "Ensure that you do not remove any information about the faculty members aside from phone numbers. As a rule of thumb, never remove any actual words, just strings of characters and numbers, UNLESS the words are part of an address. If you see an address, remove it. "},
    {"role": "user", "content": "Your must parse through the entire provided dataset. If you remove ANY usable information, the pipeline WILL break."},
    {"role": "user", "content": "You  must ensure that no data is deleted or lost when you format it."},
    {"role": "user", "content": "Watch for anything that resembles the word trustee. This is a critical word, and deleting it would damage the pipeline. For any string of text that is misspelled, but resembles trustee, correct the typo, and leave the word trustee in the data."},
    {"role": "user", "content": "Remember, if you remove ANY actual information, the process will break. You are simply formatting the text and removing phone numbers, email-addresses, and typos. You must not remove any other information, especially names of faculty members."},
    {"role": "assistant", "content": "Here is the raw data: "},
    {"role": "assistant", "content": processedPathStr},
    {"role": "assistant", "content": "\"\"\" "}
  ],
  temperature = 0.0
)

cleaned_text = completion.choices[0].message.content
print(cleaned_text)


#Remove extraneous information
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are part of a data processing pipeline. You will be given a dataset containing university faculty members, and information about these faculty. Your task is to remove any phone numbers and email addresses that appear (Or any string containing at least 7 numbers), any strings of characters that are gibberish, and any non-ASCII characters."},
#     {"role": "user", "content": "Ensure that you do not remove any information about the faculty members aside from phone numbers. As a rule of thumb, never remove any actual words, just strings of characters and numbers, UNLESS the words are part of an address. If you see an address, remove it. "},
#     {"role": "user", "content": "Your must parse through the entire provided dataset. If you remove ANY usable information, the pipeline WILL break."},
#     {"role": "user", "content": "You  must ensure that no data is deleted or lost when you format it."},
#     {"role": "user", "content": "Watch for anything that resembles the word trustee. This is a critical word, and deleting it would damage the pipeline. For any string of text that is misspelled, but resembles trustee, correct the typo, and leave the word trustee in the data."},
#     {"role": "user", "content": "Remember, if you remove ANY actual information, the process will break. You are simply formatting the text and removing phone numbers, email-addresses, and typos. You must not remove any other information, especially names of faculty members."},
#     {"role": "assistant", "content": "Here is the raw data: "},
#     {"role": "assistant", "content": processedPathStr},
#     {"role": "assistant", "content": "\"\"\" "}
#   ],
#   temperature = 0.0
# )

# cleaned_text = completion.choices[0].message.content
# print(cleaned_text)

Adelphi University —
1 South Avenue, Garden City, NY 11530-0701 .
Internet: wwvwwadelphi, edu
Enrollment: 8,000 Faculty: 900 Year Founded: 1896.

Dr. Robert Allyn Scott
President _
Education: Bucknell BA; Cornell 1976 PhD ~
Career: Associate Dean and Senior Administrator,
College of Arts and Sciences, Cornell University:

President and Chief Executive Officer, Ramapo
College (1985-2000):

Administration
President Dr. Robert Allyn Scott*® ................... 
Provost and Senior Vice President, Academic Affairs
Dr. Gayle D. Insler.... 0... eee ees 

Education: SUNY (Stony Brook) BS; UMass (Amherst) PhD
Senior Vice President, Finance and Treasurer
Education: SUNY (Albany) 1978 BS 
Vice President, Communications Lori A. Duggan- Gold. . 
Education: NYU 1987 BA
Vice President, Enrollment Management and Student oe
Success Dr. Lauren H. Mounty .................05. 
Education: South Carolina BA, MEd: Fordham PhD
Vice President, University Advancement c,
Christian P Vaupel.............----- ene 

In [185]:
# Structure the data
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are part of a data processing pipeline. You will be given a dataset containing university faculty members, and information about these faculty. Your task is to format this data such that you can extract information from the dataset more reliably and accurately later on. You must identify every faculty member within the dataset, and isolate their name and their relevant information, and return the dataset with each faculty member and their information separated."},
    {"role": "user", "content": "You must parse through the entire provided dataset. If you remove ANY usable information, the pipeline WILL break."},
    {"role": "user", "content": "You must ensure that no data is deleted or lost when you format it."},
    {"role": "user", "content": "It is possible that there is a large gap between one faculty member and the next faculty member. This does NOT mean to stop parsing and formatting the data. You must format the data until you reach the END of the provided string."},
    {"role": "user", "content": "The formatting of the data should ALWAYS be like this: Name of faculty member. *New line* Title of Faculty Member *New Line* Education of Faculty Member (If applicable) *New Line* Affiliation of Faculty Member (If applicable) *New Line* Then any other information associated with that member. Always add a gap in lines between faculty members."},
    # {"role": "user", "content": "The Affiliation of a faculty member will not always be listed. However, when it is listed, it will ALWAYS be introduced with the word: \"Affiliation:\". If you see this, be sure to put what follows the phrase for the corresponding faculty member." },
    {"role": "user", "content": "Remember, if you remove ANY actual information, the process will break. You are simply formatting the text. You must not remove any other information, especially names of faculty members."},
    {"role": "assistant", "content": "Here is the data: "},
    {"role": "assistant", "content": cleaned_text},
    {"role": "assistant", "content": "\"\"\" "}
  ],
  temperature = 0.0
)

cleaned_text = completion.choices[0].message.content
print(cleaned_text)

Dr. Robert Allyn Scott
Title: President
Education: Bucknell BA; Cornell 1976 PhD
Career: Associate Dean and Senior Administrator, College of Arts and Sciences, Cornell University; President and Chief Executive Officer, Ramapo College (1985-2000)

Dr. Gayle D. Insler
Title: Provost and Senior Vice President, Academic Affairs
Education: SUNY (Stony Brook) BS; UMass (Amherst) PhD

Lori A. Duggan-Gold
Title: Vice President, Communications
Education: NYU 1987 BA

Dr. Lauren H. Mounty
Title: Vice President, Enrollment Management and Student Success
Education: South Carolina BA, MEd; Fordham PhD

Christian P. Vaupel
Title: Vice President, University Advancement
Education: Adelphi 1996 BS, 2003 MS

Audrey B. Blumberg
Title: Senior Associate Provost, Academic Affairs

Lester B. Baltimore
Title: Associate Provost, Academic Affairs, Academic Services and Retention

Lawrence Hobbie
Title: Associate Provost, Administration

Dr. Perry E. Greene
Title: Associate Provost, Faculty Affairs

Laura Martin

In [63]:
system_message = """\
You have been assigned the role of a data extractor. Your task is to process unstructured text and extract precise and \
structured information about faculty members mentioned in the text. The text will contain details about many faculty members at U.S. Universities.

### Your Responsibilities:
1. **Read and Analyze Text:**
   Carefully read the provided unstructured text and identify any information related to faculty members, such as their names, \
job titles, education, and affiliations.


2. **Handle Ambiguities:**
   In cases where the text contains ambiguities or conflicting information, use your best judgement to resolve them and \
extract the most accurate and relevant information.

3. **Validate Information:**
   Ensure that the extracted information is valid, relevant, and conforms to the specified formats in the schema. \
For instance, the founding dates should be in recognizable date formats, and the names should be properly capitalized.

### Steps to Process:
1. **Tokenization:**
   Tokenize the input text into words or phrases that represent meaningful entities, such as names, job titles, or education, or affiliations.

2. **Entity Recognition:**
   Identify and classify entities in the text, focusing on those related to faculty members, and extract them.

3. **Entity Linking:**
   Resolve the extracted entities to their canonical forms, linking them to known entities if possible. \
   

4. **Information Structuring:**
   Organize the extracted entities under the appropriate keys in the JSON object, following the schema accurately.

5. **Validation:**
   Review the structured information to ensure its accuracy and adherence to the schema.

6. **Continuity:**
    Ensure that no information was deleted from the original text. At no point should information be removed. The only task is to format it and handle ambiguities.

### Preparation for Function Call:
Once you have processed the text and structured the information, prepare the data for the function call. Create a text \
object as specified in the schema, filling in the extracted information in the appropriate fields. \
Ensure that the text object is well-formed and adheres strictly to the schema's structure and format. \
You should separate each faculty member and their corresponding information with a line between separating two members. \
Any titles that are associated with a faculty member should be placed after the name, even if the extracted title was originally \
found before the name of the faculty member was listed.

Remember, accuracy and attention to detail are crucial in this task. Ensure that the structured information you extract is a \
true representation of the details mentioned in the unstructured text. Your role is pivotal in converting raw, unstructured data into meaningful, \
structured information that can be easily understood and utilized by other systems.
"""

In [23]:
name_message = """\
You have been assigned the role of a data extractor. Your task is to process unstructured text and extract precise and \
structured information about faculty members mentioned in the text. The text will contain details about many faculty members at U.S. Universities.

### Your Responsibilities:
1. **Read and Analyze Text:**
   Carefully read the provided unstructured text and identify any occurences of faculty member names within the text.


2. **Handle Ambiguities:**
   In cases where the text contains ambiguities or conflicting information, use your best judgement to resolve them and \
extract the most accurate and relevant information.

3. **Validate Information:**
   Ensure that the extracted information is valid, relevant, and conforms to the specified formats in the schema. \
For instance, the founding dates should be in recognizable date formats, and the names should be properly capitalized.

### Steps to Process:
1. **Tokenization:**
   Tokenize the input text into words or phrases that represent meaningful entities, specifically names.

2. **Entity Recognition:**
   Identify and classify entities in the text, focusing on those related to faculty members, and extract them.

3. **Entity Linking:**
   Resolve the extracted entities to their canonical forms, linking them to known entities if possible.

4. **Validation:**
   Review the structured information to ensure its accuracy and adherence to the schema.


### Preparation for Function Call:
Once you have processed the text and structured the information, prepare the data for the function call. Create a text \
object as specified in the schema, returning a list of ONLY names of faculty members that appear in this dataset. \
Ensure that the text object is well-formed and adheres strictly to the schema's structure and format. \

Remember, accuracy and attention to detail are crucial in this task. Ensure that the structured information you extract is a \
true representation of the details mentioned in the unstructured text. Your role is pivotal in converting raw, unstructured data into meaningful, \
structured information that can be easily understood and utilized by other systems.
"""

In [186]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    # response_format = {"type": "text_object"},
    messages=[
        # {"role": "system", "content": str(extract_information_schema)},
        {"role": "system", "content": name_message},
        {"role": "system", "content": "Ensure that you return information for EVERY individual within the dataset."},
        {"role": "system", "content": "Do not return a json"},
        {"role": "user", "content": cleaned_text},
    ],
    temperature=.1,
    # top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    # seed=1001
)

nameList = response.choices[0].message.content
print(nameList)

Dr. Robert Allyn Scott
Dr. Gayle D. Insler
Lori A. Duggan-Gold
Dr. Lauren H. Mounty
Christian P. Vaupel
Audrey B. Blumberg
Lester B. Baltimore
Lawrence Hobbie
Dr. Perry E. Greene
Laura Martin
Esther Goodcuff
Robert L. DeCarlo
Lisa S. Araujo
Christine Dolan Murphy
Eugene Palmia
Richard Garner
Jeffrey A. Kessler
Shawn W. O'Riley
Brian Lym
Jack Chen
Polly Campbell Schmitz
Thomas J. Ward
Nava Lerer
Daniel P. McCabe
Maggie Yoon Grafer
Kali Chan Peete
Dr. Sam Grogan
Dr. Anthony F. Libertella
Dr. Jane Ashdown
Patrick Coonan
Dr. Andrew W. Safyer
Dr. Jacques P. Barber
Z. Paul Akan
Frank Angello
Lois Phifer Betts
Jeffrey Bolton
Loretta V. Cangialosi
Philip DiSanto


In [187]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format = {"type": "json_object"},
    messages=[
        # {"role": "system", "content": str(extract_information_schema)},
        {"role": "system", "content": system_message},
        {"role": "system", "content": "Make the outside key: \"faculty_members. The inner keys should be: \"name, job_title, education, affiliation\"."},
        # {"role": "system", "content": "Here is a list of names faculty members that you should be returning. It is possible this list contains inaccuracies, but ensure that you check specifically for these names during the process:"},
        # {"role": "system", "content": nameList},
        {"role": "user", "content": "Rules for Determining Entities: Job Titles always appear near a name without a phrase introducting them, and should NEVER be blank. Affiliation is always introduced with the phrase \"Affiliation:\". Likewise, education is always introduced with the phrase \"Education:\"."},
        {"role": "system", "content": "Ensure that you return information for EVERY individual within the dataset."},
        {"role": "user", "content": cleaned_text},
    ],
    temperature=.1,
    # top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    seed=1001
)

jsonOutput = response.choices[0].message.content
print(jsonOutput)

{
  "faculty_members": [
    {
      "name": "Dr. Robert Allyn Scott",
      "job_title": "President",
      "education": "Bucknell BA; Cornell 1976 PhD",
      "affiliation": "Associate Dean and Senior Administrator, College of Arts and Sciences, Cornell University; President and Chief Executive Officer, Ramapo College (1985-2000)"
    },
    {
      "name": "Dr. Gayle D. Insler",
      "job_title": "Provost and Senior Vice President, Academic Affairs",
      "education": "SUNY (Stony Brook) BS; UMass (Amherst) PhD"
    },
    {
      "name": "Lori A. Duggan-Gold",
      "job_title": "Vice President, Communications",
      "education": "NYU 1987 BA"
    },
    {
      "name": "Dr. Lauren H. Mounty",
      "job_title": "Vice President, Enrollment Management and Student Success",
      "education": "South Carolina BA, MEd; Fordham PhD"
    },
    {
      "name": "Christian P. Vaupel",
      "job_title": "Vice President, University Advancement",
      "education": "Adelphi 1996 BS, 2003 

In [174]:
# Load the JSON data from the string
data = json.loads(jsonOutput)

# Extract the faculty members into a list
faculty_members = data['faculty_members']
# Print each member's information
# for member in faculty_members:
#     print(member)


# Keys to check and populate if missing
required_keys = ['job_title', 'education', 'affiliation']

# Iterate over each faculty member
for member in faculty_members:
    # Check for each required key
    for key in required_keys:
        # If the key is not present, set it to "n/a"
        if key not in member:
            member[key] = "n/a"

# Print the modified list to verify changes
for member in faculty_members:
    print(member)

{'name': 'Dr. Robert D. Hradsky', 'job_title': 'Assistant Vice President, Campus Life and Dean of Students', 'education': 'Loyola Col (MD) BBA; Vermont; Pennsylvania 2007 EdD', 'affiliation': 'American University'}
{'name': 'Abbey Silberman Fagin', 'job_title': 'Assistant Vice President, Development', 'affiliation': 'American University', 'education': 'n/a'}
{'name': 'Patricia Kelshian', 'job_title': 'Assistant Vice President, Risk Management and Safety Services', 'affiliation': 'American University', 'education': 'n/a'}
{'name': 'Cathy Hubbs', 'job_title': 'Chief Information Security Officer', 'education': 'Strayer U BS; George Mason MEd', 'affiliation': 'American University'}
{'name': 'Gihan Fernando', 'job_title': 'Executive Director, Career Center', 'affiliation': 'American University', 'education': 'n/a'}
{'name': 'Tiffany Speaks', 'job_title': 'Senior Director, Diversity and Inclusion', 'education': 'Duke BA: Stanford MA; Columbia MS', 'affiliation': 'American University'}
{'name

In [175]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

def normalize_text(text):
    # Lowercase and remove non-alphanumeric characters except spaces
    return re.sub(r'[^a-z0-9\s]', '', text.lower())


def find_incorrect_titles(entries, text, valid_titles, proximity_limit=70, threshold=85):
    # Normalize the main text for consistent matching
    normalized_text = normalize_text(text)
    words = normalized_text.split()
    segments = [' '.join(words[i:i + 12]) for i in range(len(words) - 11)]

    incorrect_titles = []
    possible_incorrect_names = []

    for entry in entries:
        normalized_name = normalize_text(entry['name'])
        normalized_job_title = normalize_text(entry['job_title'])

        # Use fuzzy matching to find the best segment that matches the name
        best_matches = process.extractOne(normalized_name, segments, scorer=fuzz.partial_ratio, score_cutoff=threshold)

        if not best_matches:
            print(f"Warning: No close matches found for name '{entry['name']}' in the text.")
            continue
        
        best_segment, match_score = best_matches
        segment_index = segments.index(best_segment)
        start_segment = max(0, segment_index - 1)
        end_segment = min(len(segments), segment_index + 2)
        extended_context = ' '.join(segments[start_segment:end_segment])

        # Check for job title in the extended context with leniency for typos
        if fuzz.partial_ratio(normalized_job_title, extended_context) < threshold:
            incorrect_titles.append({
                'name': entry['name'],
                'job_title': entry['job_title'],
                'context': extended_context,
                'reason': f"Job title '{entry['job_title']}' not clearly supported by context."
            })
            possible_incorrect_names.append(entry['name'])

    return incorrect_titles, possible_incorrect_names




def validate_titles(entries, common_title_words, original_data, proximity_limit):
    results = {}
    for entry in entries:
        # Check if any of the common title words are in the job title
        if any(word in entry['job_title'].lower() for word in common_title_words):
            context_result = find_context(entry['name'], entry['job_title'], original_data, proximity_limit)
            if context_result:
                results[entry['name']] = context_result
        else:
            results[entry['name']] = "Job title does not contain valid keywords"
    return results

In [176]:
entries = faculty_members
proximity_limit = 120
wrongnames = []
valid_titles = ["president", "provost", "senior vice president", "vice president", "treasurer", "dean", "associate provost", "administrator", "regent", "trustee", "chairman", "director", "controller", "secretary", "ex officio", "librarian", "officer", "executive assistant", "member", "student regent", "student", "counsel", "representative", "faculty", "board member", "vice chair", "commandant", "command chief", "chief", "superintendent"]
# Process each entry

incorrect_titles, wrongnames = find_incorrect_titles(entries, cleaned_text, valid_titles)
print("Incorrect Titles Found:", incorrect_titles)
print("\n")
print(wrongnames)

Incorrect Titles Found: [{'name': 'Dr. Peter Townsend Starr', 'job_title': 'Dean', 'context': 'schools and colleges college of arts and sciences dean dr peter townsend and colleges college of arts and sciences dean dr peter townsend starr colleges college of arts and sciences dean dr peter townsend starr education', 'reason': "Job title 'Dean' not clearly supported by context."}, {'name': 'Dr. James M. Goldgeier', 'job_title': 'Dean', 'context': 'ma u washington phd school of international service dean dr james m u washington phd school of international service dean dr james m goldgeier washington phd school of international service dean dr james m goldgeier education', 'reason': "Job title 'Dean' not clearly supported by context."}, {'name': 'Claudio M. Grossman', 'job_title': 'Dean', 'context': 'ba uc berkeley ma phd washington college of law dean claudio m uc berkeley ma phd washington college of law dean claudio m grossman berkeley ma phd washington college of law dean claudio m gr

In [177]:
# response = client.chat.completions.create(
#     model="gpt-3.5-turbo-1106",
#     response_format = {"type": "json_object"},
#     messages=[
#         {"role": "system", "content": "You will be given a json object populated with information extracted from a dataset, and a list of these faculty members who potentially have job title information listed for them within the json file.\
#          Your job is to parse through every faculty member that is listed as containing potentially incorrect job titles, and to refer back to the dataset, and determine if the job title within the json was incorrect. \
#          If you find that the information was incorrect, fix the information to the correct values, and change nothing else about the structure of the json."},
#         {"role": "system", "content": "A job title is generally wrong because the title for the faculty member is being assigned a title belonging to a separate faculty member. To fix this, \
#          if the incorrect title was found after the faculty member's name, instead look for the title before their name. If the title was found before the faculty member's name, \
#          look for the title after their name."},
#         {"role": "system", "content": str(jsonOutput)},
#         {"role": "system", "content": "Make the outside key: \"faculty_members. The inner keys should be: \"name, job_title, education, affiliation\"."},
#         {"role": "system", "content": "Here is a list of names faculty members that you should be checking, and the context surrounding their names: "},
#         {"role": "system", "content": str(wrongnames)},
#         {"role": "system", "content": str(incorrect_titles)},
#         {"role": "system", "content": "."},
#         {"role": "user", "content": cleaned_text},
#     ],
#     temperature=.1,
#     # top_p=1,
#     frequency_penalty=0,
#     presence_penalty=0,
#     seed=1001
# )

response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "You are provided with a JSON object containing faculty member information. Each faculty member entry contains a name, job title, education, and affiliation. Some faculty members have incorrect job titles that need to be corrected based on additional context information."},
        {"role": "system", "content": "The job titles might be swapped or incorrectly assigned. For faculty members listed as having incorrect titles, please adjust their titles based on the context given. If the title was incorrect because it appears after their name in the dataset, the correct title should instead be the one appearing before their name, and vice versa."},
        {"role": "system", "content": json.dumps(jsonOutput)},  # Assuming jsonOutput is a dictionary
        {"role": "system", "content": "The key structure of the JSON is as follows: the main key is 'faculty_members', and the sub-keys within each faculty member object are 'name', 'job_title', 'education', and 'affiliation'."},
        {"role": "system", "content": "Here are the names of faculty members you need to review, along with the context indicating their name placement issues:"},
        {"role": "system", "content": str(wrongnames)},  # Assuming wrongnames is a list or dict
        {"role": "user", "content": "Correct the job titles in the JSON based on the instructions and context provided. Here is an example correction: if Dr. John Smith's entry initially had 'Associate Professor' listed before his name, but that was incorrect, you should change it to the title listed after his name in the dataset."},
        {"role": "system", "content": "Here is the original dataset:"},
        {"role": "user", "content": cleaned_text},
    ],
    temperature=0.6,
    frequency_penalty=0,
    presence_penalty=0,
    # seed=1001
)



help = response.choices[0].message.content
print(help)



{
  "faculty_members": [
    {
      "name": "Dr. Robert D. Hradsky",
      "job_title": "Assistant Vice President, Campus Life and Dean of Students",
      "education": "Loyola Col (MD) BBA; Vermont; Pennsylvania 2007 EdD",
      "affiliation": "American University"
    },
    {
      "name": "Abbey Silberman Fagin",
      "job_title": "Assistant Vice President, Development",
      "affiliation": "American University"
    },
    {
      "name": "Patricia Kelshian",
      "job_title": "Assistant Vice President, Risk Management and Safety Services",
      "affiliation": "American University"
    },
    {
      "name": "Cathy Hubbs",
      "job_title": "Chief Information Security Officer",
      "education": "Strayer U BS; George Mason MEd",
      "affiliation": "American University"
    },
    {
      "name": "Gihan Fernando",
      "job_title": "Executive Director, Career Center",
      "affiliation": "American University"
    },
    {
      "name": "Tiffany Speaks",
      "job_title":

In [178]:
csv_name = 'jsonOutput.csv'
fieldnames = list(faculty_members[0].keys())

with open(csv_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Writing headers
    # writer.writeheader()

    # Writing data
    for person in faculty_members:
        writer.writerow(person)

print(f"Data successfully written to {csv_name}")

Data successfully written to jsonOutput.csv


In [102]:
for x in processedPath[9]:
    print(x)

 

Professional Schools and Colleges

School of Architecture, Art and Design

Dean Carey Carscallen............... 00 ee eee eee eee (269) 471-6250
E-mail: ecarey@andrews.edu . . Fax: (269) 471-6261
Education: Idaho 1995 BArch, 1996 MArch

College of Arts and Sciences

Dean Dr. Keith E. Mattingly .............. eee eee ae (269) 471-3411

E-mail: matt@andrews.edu Fax: (269) 471-6236
Education: Andrews 1997 PhD

School of Business Administration

Dean Dr. Allen Stembridge..................00.008. (269) 471-3632
E-mail: stem@andrews.edu Fax: (269) 471-6158

Education: U South Africa: Andrews 1982 MBA, 1984 EdD

School of Education

Dean Dr. James R. “Jim” Jeffery ..............0.... (269) 471-3481
E-mail: jimjeff@andrews.edu Fax: (269) 471-6374
Education: Columbia Union 1971 BA: Andrews 1973 MA;
Alberta 1990 PhD

School of Health Professions
Dean Dr. Emmanuel Rudatsikira .................05- (269) 471-777)
Education: Loma Linda PhD

SDA Theological Seminary .

Dean Dr. Jiri Moskala 2.0.0.

In [103]:
def clean_data(data):
    # List of unwanted substrings, case insensitive
    unwanted_substrings = ["education:", "affiliation:", "career:", "e-mail:", "tel:", "fax:"]

    # Filter the list
    filtered_data = [
        element for element in data
        if element.strip() and  # Check if the element is not just whitespace
        not any(substring.lower() in element.lower() for substring in unwanted_substrings)
    ]

    return filtered_data



# Clean the data
cleaned_list = clean_data(processedPath[9])

# Output the cleaned list
print("Cleaned Data List:")
for item in cleaned_list:
    print(item)

Cleaned Data List:
Professional Schools and Colleges
School of Architecture, Art and Design
Dean Carey Carscallen............... 00 ee eee eee eee (269) 471-6250
College of Arts and Sciences
Dean Dr. Keith E. Mattingly .............. eee eee ae (269) 471-3411
School of Business Administration
Dean Dr. Allen Stembridge..................00.008. (269) 471-3632
School of Education
Dean Dr. James R. “Jim” Jeffery ..............0.... (269) 471-3481
Alberta 1990 PhD
School of Health Professions
Dean Dr. Emmanuel Rudatsikira .................05- (269) 471-777)
SDA Theological Seminary .
Dean Dr. Jiri Moskala 2.0.0.0... 0.0. cece eee eee (269) 471-3205
School of Graduate Studies and Research
Dean Dr. Christon Arthur ..........0.0 20 cece eee eee (269) 471-3405
Board of Trustees
Niels-Erik A. Andreasen ...........0 00+ e ee eee Secretary of the Board
Dr. Lisa Ml. Beardsley-Hardy .......... 06. c cee eee eee renee Trustee
Claremont Grad 2006 MBA
University
Rose Marie Cazeau, RN...........-.6: cec

In [110]:
singlePage = cleaned_list
singlePage = removeEmptyElements(singlePage)
singlePage = removeEmptyElements(singlePage)
titleIndices = tagNames(singlePage, titleIndices)
titleIndices = breakChunks(singlePage, titleIndices)
titleIndices = breakChunks(singlePage, titleIndices)
titleIndices = breakDoubleNames(singlePage, titleIndices)
titleIndices = breakTwoLineDoubleName(singlePage, titleIndices)
# titleIndices = connectSingleLines(singlePage, titleIndices)
printFormattedText(singlePage, titleIndices)

Professional Schools and Colleges



School of Architecture, Art and Design
Dean Carey Carscallen............... 00 ee eee eee eee (269) 471-6250



College of Arts and Sciences



Dean Dr. Keith E. Mattingly .............. eee eee ae (269) 471-3411



School of Business Administration



Dean Dr. Allen Stembridge..................00.008. (269) 471-3632



School of Education



Dean Dr. James R. “Jim” Jeffery ..............0.... (269) 471-3481



Alberta 1990 PhD



School of Health Professions



Dean Dr. Emmanuel Rudatsikira .................05- (269) 471-777)



SDA Theological Seminary .



Dean Dr. Jiri Moskala 2.0.0.0... 0.0. cece eee eee (269) 471-3205



School of Graduate Studies and Research



Dean Dr. Christon Arthur ..........0.0 20 cece eee eee (269) 471-3405



Board of Trustees



Niels-Erik A. Andreasen ...........0 00+ e ee eee Secretary of the Board



Dr. Lisa Ml. Beardsley-Hardy .......... 06. c cee eee eee renee Trustee



Claremont Grad 2006 MBA
University



Ro

In [111]:
def insert_at_indices(source_list, indices, text_to_insert):
    # Sort indices in ascending order to handle multiple insertions correctly
    sorted_indices = sorted(indices)
    # Adjust indices to account for the lengthening of the list as items are inserted
    for i, index in enumerate(sorted_indices):
        # Adjust the index by adding the current position in the list of indices
        # because each insertion increases the length of the list
        adjusted_index = index + i
        source_list.insert(adjusted_index, text_to_insert)
    return source_list

text = "\n"

# Modify the string_list
singlePage = insert_at_indices(singlePage, titleIndices, text)
for x in singlePage:
    print(x)

Professional Schools and Colleges




School of Architecture, Art and Design
Dean Carey Carscallen............... 00 ee eee eee eee (269) 471-6250








College of Arts and Sciences








Dean Dr. Keith E. Mattingly .............. eee eee ae (269) 471-3411








School of Business Administration








Dean Dr. Allen Stembridge..................00.008. (269) 471-3632








School of Education












Dean Dr. James R. “Jim” Jeffery ..............0.... (269) 471-3481








Alberta 1990 PhD




School of Health Professions




















Dean Dr. Emmanuel Rudatsikira .................05- (269) 471-777)












SDA Theological Seminary .








Dean Dr. Jiri Moskala 2.0.0.0... 0.0. cece eee eee (269) 471-3205












School of Graduate Studies and Research




















Dean Dr. Christon Arthur ..........0.0 20 cece eee eee (269) 471-3405




Board of Trustees






















Niels-Erik A. Andreasen ...........0 00+ e ee eee Secretary of the Board


In [112]:
def clean_data(data):
    # List of unwanted substrings, case insensitive
    unwanted_substrings = ["fadsfdas"]

    # Filter the list
    filtered_data = [
        element for element in data
        if not any(substring.lower() in element.lower() for substring in unwanted_substrings)
    ]

    return filtered_data



# Clean the data
singlePage = clean_data(singlePage)

for x in singlePage:
    print(x)

Professional Schools and Colleges




School of Architecture, Art and Design
Dean Carey Carscallen............... 00 ee eee eee eee (269) 471-6250








College of Arts and Sciences








Dean Dr. Keith E. Mattingly .............. eee eee ae (269) 471-3411








School of Business Administration








Dean Dr. Allen Stembridge..................00.008. (269) 471-3632








School of Education












Dean Dr. James R. “Jim” Jeffery ..............0.... (269) 471-3481








Alberta 1990 PhD




School of Health Professions




















Dean Dr. Emmanuel Rudatsikira .................05- (269) 471-777)












SDA Theological Seminary .








Dean Dr. Jiri Moskala 2.0.0.0... 0.0. cece eee eee (269) 471-3205












School of Graduate Studies and Research




















Dean Dr. Christon Arthur ..........0.0 20 cece eee eee (269) 471-3405




Board of Trustees






















Niels-Erik A. Andreasen ...........0 00+ e ee eee Secretary of the Board


In [90]:
system_message = """\
You have been assigned the role of a data extractor. Your task is to process unstructured text and extract precise and \
structured information about faculty members mentioned in the text. The text will contain details about many faculty members at U.S. Universities.

### Your Responsibilities:
1. **Read and Analyze Text:**
   Carefully read the provided unstructured text and identify any information related to faculty members, specifically, their names, \
and job titles.


2. **Handle Ambiguities:**
   In cases where the text contains ambiguities or conflicting information, use your best judgement to resolve them and \
extract the most accurate and relevant information.

3. **Validate Information:**
   Ensure that the extracted information is valid, relevant, and conforms to the specified formats in the schema. \
For instance, the founding dates should be in recognizable date formats, and the names should be properly capitalized.

### Steps to Process:
1. **Tokenization:**
   Tokenize the input text into words or phrases that represent meaningful entities, such as names and job titles

2. **Entity Recognition:**
   Identify and classify entities in the text, focusing on those related to faculty members, and extract them.

3. **Entity Linking:**
   Resolve the extracted entities to their canonical forms, linking them to known entities if possible. \
   

4. **Information Structuring:**
   Organize the extracted entities under the appropriate keys in the JSON object, following the schema accurately.

5. **Validation:**
   Review the structured information to ensure its accuracy and adherence to the schema.

6. **Continuity:**
    Ensure that no information was deleted from the original text. At no point should information be removed. The only task is to format it and handle ambiguities. You must parse through the entire dataset. No faculty members may be left out.

### Preparation for Function Call:
Once you have processed the text and structured the information, prepare the data for the function call. Create a text \
object as specified in the schema, filling in the extracted information in the appropriate fields. \
Ensure that the text object is well-formed and adheres strictly to the schema's structure and format. \
You should separate each faculty member and their corresponding information with a line between separating two members. \
Any titles that are associated with a faculty member should be placed after the name, even if the extracted title was originally \
found before the name of the faculty member was listed.

Remember, accuracy and attention to detail are crucial in this task. Ensure that the structured information you extract is a \
true representation of the details mentioned in the unstructured text. Your role is pivotal in converting raw, unstructured data into meaningful, \
structured information that can be easily understood and utilized by other systems.
"""

In [113]:
def extract_names(data):
    name_list = []
    for text in data:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                name_list.append(ent.text)
    return name_list

extracted_names = extract_names(singlePage)
for x in extracted_names:
    print(x)

Dean Carey Carscallen
Dean
Keith E. Mattingly
Dean
Allen Stembridge
Dean
James R. “Jim” Jeffery
Dean
Emmanuel Rudatsikira
Dean
Jiri Moskala
Dean
Christon Arthur
Niels-Erik A. Andreasen
Lisa Ml
Marie Cazeau
J. Christensen
Jerome L. Davis
Michael G. Edge
Jay Gallimore
Cletus R. Georges... 1.
Lynn S. Gray
Loren Hamel
Edwin Hernandez
Dann Hotelling
Van G. Hurst 20000
Daniel R. Jackson
Oshawa
Mark A. Johnson
ccc ccc
Sandra K. Johnson
Donald G. King
Robert E. Lemon
Columbia Pike
Donald E. Livesay
Berrien Springs
Fred M. Manchur
Larry R. Moore
William G. C. Murdoch
Jr
G. T. Ng
PAD
Ray Pichette
Juan R. Prestol
Barbara J. Randall
David R. Rowe
Benjamin D. Schoun
Glynn C.W. Scott
Dolores E. Slikkers
Ron C. Smith
Paul A. Stokstad
ene
Judith L. “Judy” Storfjell
Garry Sudds
bec
Gary F Thurber
Lynne K. Waihee
Dana L. Wales
Rebekah May
Wang Cheng
Ted N. C. Wilson
Edward E. Zinke


In [114]:
response = client.chat.completions.create(
    model="gpt-4",
    # response_format = {"type": "text_object"},
    messages=[
        # {"role": "system", "content": str(extract_information_schema)},
        {"role": "system", "content": name_message},
        {"role": "system", "content": "Ensure that you return every faculty member listed in the dataset."},
        {"role": "system", "content": "Do not return a json"},
        {"role": "user", "content": str(singlePage)},
        # {"role": "user", "content": str(processedPathStr[8])},
    ],
    temperature=.1,
    # top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    # seed=1001
)


nameList = response.choices[0].message.content
print(nameList)

The faculty members mentioned in the text are:

1. Carey Carscallen
2. Dr. Keith E. Mattingly
3. Dr. Allen Stembridge
4. Dr. James R. “Jim” Jeffery
5. Dr. Emmanuel Rudatsikira
6. Dr. Jiri Moskala
7. Dr. Christon Arthur
8. Niels-Erik A. Andreasen
9. Dr. Lisa Ml. Beardsley-Hardy
10. Rose Marie Cazeau, RN
11. Debbi J. Christensen
12. Jerome L. Davis
13. Michael G. Edge
14. Jay Gallimore
15. Cletus R. Georges
16. Lynn S. Gray
17. Loren Hamel
18. Edwin Hernandez
19. Dann Hotelling
20. Van G. Hurst
21. Daniel R. Jackson
22. Mark A. Johnson
23. Sandra K. Johnson
24. Donald G. King
25. Robert E. Lemon
26. Donald E. Livesay
27. Fred M. Manchur
28. Larry R. Moore
29. William G. C. Murdoch, Jr.
30. Dr. G. T. Ng, PAD
31. Ray Pichette
32. Juan R. Prestol
33. Barbara J. Randall
34. David R. Rowe
35. Benjamin D. Schoun
36. Glynn C.W. Scott
37. Dolores E. Slikkers
38. Dr. Ron C. Smith, PhD
39. Paul A. Stokstad
40. Dr. Judith L. “Judy” Storfjell, RN
41. Garry Sudds
42. Gary F Thurber
43. Lynne K. Waihe

In [115]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format = {"type": "json_object"},
    messages=[
        # {"role": "system", "content": str(extract_information_schema)},
        {"role": "system", "content": system_message},
        {"role": "system", "content": "Make the outside key: \"faculty_members. The inner keys should be: \"name, job_title."},
        # {"role": "system", "content": "Here is a list of names faculty members that you should be returning. It is possible this list contains inaccuracies, but ensure that you check specifically for these names during the process:"},
        # {"role": "system", "content": nameList},
        {"role": "user", "content": "Job Titles always appear near a name without a phrase introducting them."},
        {"role": "system", "content": "Ensure that you return information for EVERY individual within the dataset."},
        {"role": "user", "content": str(singlePage)},
        {"role": "system", "content": "Here is the list of names to extract information for: "},
        {"role": "user", "content": str(nameList)},
    ],
    temperature=.1,
    # top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    # seed=1001
)

jsonOutput = response.choices[0].message.content
print(jsonOutput)

{
  "faculty_members": [
    {
      "name": "Carey Carscallen",
      "job_title": "Dean"
    },
    {
      "name": "Dr. Keith E. Mattingly",
      "job_title": "Dean"
    },
    {
      "name": "Dr. Allen Stembridge",
      "job_title": "Dean"
    },
    {
      "name": "Dr. James R. “Jim” Jeffery",
      "job_title": "Dean"
    },
    {
      "name": "Dr. Emmanuel Rudatsikira",
      "job_title": "Dean"
    },
    {
      "name": "Dr. Jiri Moskala",
      "job_title": "Dean"
    },
    {
      "name": "Dr. Christon Arthur",
      "job_title": "Dean"
    },
    {
      "name": "Niels-Erik A. Andreasen",
      "job_title": "Secretary of the Board"
    },
    {
      "name": "Dr. Lisa Ml. Beardsley-Hardy",
      "job_title": "Trustee"
    },
    {
      "name": "Rose Marie Cazeau, RN",
      "job_title": "Trustee"
    },
    {
      "name": "Debbi J. Christensen",
      "job_title": "Trustee"
    },
    {
      "name": "Jerome L. Davis",
      "job_title": "Trustee"
    },
    {
     

In [116]:
# Load the JSON data from the string
data = json.loads(jsonOutput)

# Extract the faculty members into a list
faculty_members = data['faculty_members']
# Print each member's information
# for member in faculty_members:
#     print(member)


# Keys to check and populate if missing
required_keys = ['name','job_title']

# Iterate over each faculty member
for member in faculty_members:
    # Check for each required key
    for key in required_keys:
        # If the key is not present, set it to "n/a"
        if key not in member:
            member[key] = "n/a"

# Print the modified list to verify changes
for member in faculty_members:
    print(member)

{'name': 'Carey Carscallen', 'job_title': 'Dean'}
{'name': 'Dr. Keith E. Mattingly', 'job_title': 'Dean'}
{'name': 'Dr. Allen Stembridge', 'job_title': 'Dean'}
{'name': 'Dr. James R. “Jim” Jeffery', 'job_title': 'Dean'}
{'name': 'Dr. Emmanuel Rudatsikira', 'job_title': 'Dean'}
{'name': 'Dr. Jiri Moskala', 'job_title': 'Dean'}
{'name': 'Dr. Christon Arthur', 'job_title': 'Dean'}
{'name': 'Niels-Erik A. Andreasen', 'job_title': 'Secretary of the Board'}
{'name': 'Dr. Lisa Ml. Beardsley-Hardy', 'job_title': 'Trustee'}
{'name': 'Rose Marie Cazeau, RN', 'job_title': 'Trustee'}
{'name': 'Debbi J. Christensen', 'job_title': 'Trustee'}
{'name': 'Jerome L. Davis', 'job_title': 'Trustee'}
{'name': 'Michael G. Edge', 'job_title': 'Trustee'}
{'name': 'Jay Gallimore', 'job_title': 'Trustee'}
{'name': 'Cletus R. Georges', 'job_title': 'Trustee'}
{'name': 'Lynn S. Gray', 'job_title': 'Trustee'}
{'name': 'Loren Hamel', 'job_title': 'Trustee'}
{'name': 'Edwin Hernandez', 'job_title': 'Trustee'}
{'name'

In [100]:
csv_name = 'jsonOutput.csv'
fieldnames = list(faculty_members[0].keys())

with open(csv_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Writing headers
    # writer.writeheader()

    # Writing data
    for person in faculty_members:
        writer.writerow(person)

print(f"Data successfully written to {csv_name}")

Data successfully written to jsonOutput.csv
